## Decision Tree Classification

A decision tree uses an upside down tree structure to represent a number of possible decision paths and an outcome for each path.

### Star classification data

This data was found on Kaggle. It consists of several variables used to classify stars. For more information you can find the dataset [here](https://www.kaggle.com/brsdincer/star-type-classification).

Variables:
- Temperature -- K
- L -- L/Lo: Lo = 3.828 x 10^26 Watts (Avg Luminosity of Sun)
- R -- R/Ro: Ro = 6.9551 x 10^8 m(Avg Radius of Sun)
- AM -- Mv
- Color -- General Color of Spectrum
- Spectral_Class -- O,B,A,F,G,K,M / SMASS - https://en.wikipedia.org/wiki/Asteroid_spectral_types
- Type -- Red Dwarf, Brown Dwarf, White Dwarf, Main Sequence , Super Giants, Hyper Giants

TARGET: Star Type from 0 to 5
- Red Dwarf - 0
- Brown Dwarf - 1
- White Dwarf - 2
- Main Sequence - 3
- Super Giants - 4
- Hyper Giants - 5

We will be implementing the algorithm found [here](https://www.youtube.com/watch?v=sgQAhG5Q7iY).
___

In this notebook we will use the following libraries:
- [numpy](https://numpy.org/doc/stable/)
- [pandas](https://pandas.pydata.org/docs/)
- sklearn
    - [model_selection.train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)
    - [metrics.accuracy_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)
___

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [12]:
data = pd.read_csv("Stars.csv")
data.head(10)

,Temperature,L,R,A_M,Color,Spectral_Class,Type
0,3068,0.002400,0.1700,16.12,Red,M,0
1,3042,0.000500,0.1542,16.60,Red,M,0
2,2600,0.000300,0.1020,18.70,Red,M,0
3,2800,0.000200,0.1600,16.65,Red,M,0
4,1939,0.000138,0.1030,20.06,Red,M,0
5,2840,0.000650,0.1100,16.98,Red,M,0
6,2637,0.000730,0.1270,17.22,Red,M,0
7,2600,0.000400,0.0960,17.40,Red,M,0
8,2650,0.000690,0.1100,17.45,Red,M,0
9,2700,0.000180,0.1300,16.05,Red,M,0


In [13]:
x = data.iloc[:, :-1].values
y = data.iloc[:, -1].values.reshape(-1, 1)
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3)

In [14]:
class Node():
    
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        # decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # leaf node
        self.value = value

In [15]:
class DecisionTreeClassifier():
    
    def __init__(self, min_samples_split=2, max_depth=2):
        # initialize root
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, data, depth=0):
        X, Y = data[:, :-1], data[:, -1]
        num_samples, num_features = np.shape(x)
        
        # split until stopping conditions are met
        if num_samples >= self.min_samples_split and depth <= self.max_depth:
            # find the best split
            best_split = self.get_best_split(data, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"] > 0:
                left_subtree = self.build_tree(best_split["left_data"], depth + 1)
                right_subtree = self.build_tree(best_split["right_data"], depth + 1)
                return Node(best_split["feature_index"], best_split["threshold"],
                            left_subtree, right_subtree, best_split["info_gain"])
            
        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)
    
    def get_best_split(self, data, num_samples, num_features):
        best_split = {}
        max_info_gain = -float("inf")
        
        for feature_index in range(num_features):
            feature_values = data[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            for threshold in possible_thresholds:
                left_data, right_data = self.split(data, feature_index, threshold)
                if len(left_data) > 0 and len(right_data) > 0:
                    y, left_y, right_y = data[:, -1], left_data[:, -1], right_data[:, -1]
                    info_gain = self.information_gain(y, left_y, right_y, "gini")
                    if info_gain > max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["left_data"] = left_data
                        best_split["right_data"] = right_data
                        best_split["info_gain"] = info_gain
                        max_info_gain = info_gain
        return best_split

    def split(self, data, feature_index, threshold):
        left_data = np.array([row for row in data if row[feature_index] <= threshold])
        right_data = np.array([row for row in data if row[feature_index] > threshold])
        return left_data, right_data
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        l_weight = len(l_child) / len(parent)
        r_weight = len(r_child) / len(parent)
        if mode == "gini":
            gain = self.gini_index(parent) - (l_weight * self.gini_index(l_child) + r_weight * self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (l_weight * self.entropy(l_child) + r_weight * self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls ** 2
        return 1 - gini
    
    def calculate_leaf_value(self, Y):
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        if not tree:
            tree = self.root
            
        if tree.value is not None:
            print(tree.value)
        
        else:
            print("X_" + str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
            
    def fit(self, X, Y):
        data = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(data)
        
    def predict(self, X):
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions
    
    def make_prediction(self, x, tree):
        if tree.value != None:
            return tree.value
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
        

In [16]:
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=5)
classifier.fit(train_x, train_y)
classifier.print_tree()

X_2 <= 0.015 ? 0.17622991225475693
 left:2
 right:X_3 <= 14.79 ? 0.21295712852108573
  left:X_1 <= 0.039 ? 0.2566186556927298
    left:1
    right:X_2 <= 10.6 ? 0.33041666666666664
        left:3
        right:X_2 <= 98.0 ? 0.49382716049382713
                left:4
                right:5
  right:0


In [17]:
prediction = classifier.predict(test_x)
accuracy_score(test_y, prediction)

0.9861111111111112

We are able to achieve 98% accuracy with our model on the test data.